In [2]:
# Imports
from html.parser import HTMLParser
import re
import requests
import urllib3
import html
import pandas as pd

In [8]:
"""
A simple parser to extract all the text of a publication from the body.
It removes any internal script and removes special characters with the str.strip function.
It also gets rid of pagination (Página nnn)
"""
class PublicationParser(HTMLParser):
    #Initializing lists
    lsStartTags = list()
    lsEndTags = list()
    lsStartEndTags = list()
    lsComments = list()
    lsData=list()

    # Indicates when we are inside the body tag
    inBody=False
    # Marker for scripts
    inScript=False

    #HTML Parser Methods
    def handle_starttag(self, startTag, attrs):
        self.lsStartTags.append(startTag)
        if(startTag=="body"):
            self.inBody=True
        if(startTag=="script"):
            self.inScript=True

    def handle_endtag(self, endTag):
        self.lsEndTags.append(endTag)
        if(endTag=="body"):
            self.inBody=False
        elif(endTag=="script"):
            self.inScript=False

    def handle_startendtag(self,startendTag, attrs):
       self.lsStartEndTags.append(startendTag)

    def handle_comment(self,data):
       self.lsComments.append(data)

    def handle_data(self, data):
        if(self.inBody and not self.inScript and data!=''):
            if(not (data.startswith('Página ') or data.startswith('(Página') )):
                clean=data.strip()
                if(clean!=''):
                    self.lsData.append(clean)


In [3]:
term="XII"
#page=f"https://www.congreso.es/publicaciones-organo?p_p_id=publicaciones&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_publicaciones_descOrg=Pleno-y-Diputacion-Permanente&_publicaciones_legislatura={term}&_publicaciones_seccion=Congreso&_publicaciones_publicacion=D"
page=f"https://www.congreso.es/publicaciones-organo?p_p_id=publicaciones&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_resource_id=filtrarListado&p_p_cacheability=cacheLevelPage&_publicaciones_seccion=Congreso&_publicaciones_legislatura={term}&_publicaciones_descOrg=Pleno-y-Diputacion-Permanente&_publicaciones_publicacion=D"

In [14]:
http = urllib3.PoolManager()
r=http.request("GET",page)
print(r.status)

200


In [14]:
import json

term="14"
pagenr=1
#page=f"https://www.congreso.es/publicaciones-organo?p_p_id=publicaciones&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_publicaciones_descOrg=Pleno-y-Diputacion-Permanente&_publicaciones_legislatura={term}&_publicaciones_seccion=Congreso&_publicaciones_publicacion=D"
pagequery=f"https://www.congreso.es/publicaciones-organo?p_p_id=publicaciones&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_resource_id=filtrarListado&p_p_cacheability=cacheLevelPage&_publicaciones_seccion=Congreso&_publicaciones_descOrg=Pleno-y-Diputacion-Permanente&_publicaciones_publicacion=D"
pageform={"_publicaciones_legislatura":term,
            "_publicaciones_comision":'',
            "_publicaciones_seccion":'',
          "_publicaciones_fromOrganos":1,
          "_publicaciones_paginaActual":pagenr
}

http = urllib3.PoolManager()

headers = {'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
           'accept':'application/json, text/javascript, */*; q=0.01'
}
#r=http.request("POST",pagequery,headers=headers,data=pageform)
r=http.request_encode_body('POST',pagequery,headers=headers,fields=pageform,encode_multipart=False)
print(r.status)
print(r.data.decode("utf-8"))


json.loads(r.data.decode("utf-8"))
#DSCD-14-PL-1-C1%20DSCD-14-PL-1.CODI.#1

200
{"documento10":{"diario":"/public_oficiales/L14/CONG/DS/PL/DSCD-14-PL-10.PDF","secc":"CONGRESO","orga":"Pleno","encabezado":"DS","fecha_mensaje":"de 19/02/2020","numdoc":"10","texto_integro":"/wc/abrirPopUp?CMD=VERDOC&CONF=BRSPUB.cnf&BASE=PU14&PIECE=PUWD&DOCS=1-1&FMT=PUWTXDTS.fmt&OPDEF=Y&QUERY=(D).PUBL.+Y+(Congreso).SECC.+Y+(Pleno-y-Diputacion-Permanente).ORSE.+Y+DSCD-14-PL-10.CODI.#1","fecha":"20200219","cve":"DSCD-14-PL-10","mensaje":"Congreso de los Diputados, Pleno y Dip. Perm., núm. 10,","seri":"Pleno y Diputación Permanente ","ndia":"10","subi":""},"documento20":{"diario":"/public_oficiales/L14/CONG/DS/PL/DSCD-14-PL-20.PDF","secc":"CONGRESO","orga":"Pleno","encabezado":"DS","fecha_mensaje":"de 29/04/2020","numdoc":"20","texto_integro":"/wc/abrirPopUp?CMD=VERDOC&CONF=BRSPUB.cnf&BASE=PU14&PIECE=PUWD&DOCS=1-1&FMT=PUWTXDTS.fmt&OPDEF=Y&QUERY=(D).PUBL.+Y+(Congreso).SECC.+Y+(Pleno-y-Diputacion-Permanente).ORSE.+Y+DSCD-14-PL-20.CODI.#1","fecha":"20200429","cve":"DSCD-14-PL-20","mensa

{'documento10': {'diario': '/public_oficiales/L14/CONG/DS/PL/DSCD-14-PL-10.PDF',
  'secc': 'CONGRESO',
  'orga': 'Pleno',
  'encabezado': 'DS',
  'fecha_mensaje': 'de 19/02/2020',
  'numdoc': '10',
  'texto_integro': '/wc/abrirPopUp?CMD=VERDOC&CONF=BRSPUB.cnf&BASE=PU14&PIECE=PUWD&DOCS=1-1&FMT=PUWTXDTS.fmt&OPDEF=Y&QUERY=(D).PUBL.+Y+(Congreso).SECC.+Y+(Pleno-y-Diputacion-Permanente).ORSE.+Y+DSCD-14-PL-10.CODI.#1',
  'fecha': '20200219',
  'cve': 'DSCD-14-PL-10',
  'mensaje': 'Congreso de los Diputados, Pleno y Dip. Perm., núm. 10,',
  'seri': 'Pleno y Diputación Permanente ',
  'ndia': '10',
  'subi': ''},
 'documento20': {'diario': '/public_oficiales/L14/CONG/DS/PL/DSCD-14-PL-20.PDF',
  'secc': 'CONGRESO',
  'orga': 'Pleno',
  'encabezado': 'DS',
  'fecha_mensaje': 'de 29/04/2020',
  'numdoc': '20',
  'texto_integro': '/wc/abrirPopUp?CMD=VERDOC&CONF=BRSPUB.cnf&BASE=PU14&PIECE=PUWD&DOCS=1-1&FMT=PUWTXDTS.fmt&OPDEF=Y&QUERY=(D).PUBL.+Y+(Congreso).SECC.+Y+(Pleno-y-Diputacion-Permanente).ORSE

In [10]:
pagenr=5
pageform={"_publicaciones_legislatura":term,
           "_publicaciones_comision":'',
           "_publicaciones_seccion":'',
          "_publicaciones_fromOrganos":1,
          "_publicaciones_paginaActual":pagenr
}

import urllib.parse

print(urllib.parse.urlencode(pageform))
"/wc/abrirPopUp?CMD=VERDOC&CONF=BRSPUB.cnf&BASE=PU12&PIECE=PU12&DOCS=1-1&FMT=PUWTXDTS.fmt&OPDEF=Y&QUERY=(D).PUBL.+Y+(Congreso).SECC.+Y+(Pleno-y-Diputacion-Permanente).ORSE.+Y+DSCD-12-PL-39.CODI.#1"
LINK="https://www.congreso.es/busqueda-de-publicaciones?p_p_id=publicaciones&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_publicaciones_mode=mostrarTextoIntegro&_publicaciones_legislatura=XIV&_publicaciones_texto=&_publicaciones_id_texto=BOCG_D_14_4_18.CODI.#1"

SyntaxError: invalid syntax (2811566127.py, line 12)